In [23]:
# idea use the pandas read_html function to match the table with the correnct name using the values that are the same in both tables and create a name mapping similar to the one for the statements

In [ ]:
accession_number = accession_number_series[0]
soup = get_statement_soup(ticker, accession_number, statement_name)

In [24]:
print_links_to_desired_statment(ticker, statement_name)

https://www.sec.gov/Archives/edgar/data/0000719955/000162828023009175/R6.htm
https://www.sec.gov/Archives/edgar/data/0000719955/000162828022007494/R6.htm
https://www.sec.gov/Archives/edgar/data/0000719955/000119312521100319/R5.htm
https://www.sec.gov/Archives/edgar/data/0000719955/000119312520088937/R5.htm
https://www.sec.gov/Archives/edgar/data/0000719955/000119312519097973/R5.htm
https://www.sec.gov/Archives/edgar/data/0000719955/000119312518102232/R5.htm
https://www.sec.gov/Archives/edgar/data/0000719955/000119312517104341/R4.htm
https://www.sec.gov/Archives/edgar/data/0000719955/000119312516525847/R4.htm
https://www.sec.gov/Archives/edgar/data/0000719955/000119312515118009/R4.htm
https://www.sec.gov/Archives/edgar/data/0000719955/000119312514129974/R4.htm
https://www.sec.gov/Archives/edgar/data/0000719955/000119312513142309/R4.htm
https://www.sec.gov/Archives/edgar/data/0000719955/000119312512140070/R3.htm
https://www.sec.gov/Archives/edgar/data/0000719955/000119312511085046/R3.xml

In [16]:
def create_column_name_mapping_to_statement(soup: BeautifulSoup) -> dict:
    """
    Create a dictionary that maps internal column titles to displayed names.

    Parameters:
    - soup (BeautifulSoup): The HTML soup object containing the balance sheet.

    Returns:
    - dict: Dictionary mapping internal column titles to displayed names.
    """
    column_name_mapping_common_as_key = {}
    column_name_mapping_gaap_as_key = {}

    # Iterate through each row in the balance sheet table
    for row in soup.select("tr.re, tr.ro, tr.reu, tr.rou"):
        # Extract the 'onclick' attribute to get the internal column title
        onclick_attr = row.select_one("td.pl a, td.pl.custom a")["onclick"]
        gaap_tag = onclick_attr.split("defref_")[-1].split("',")[0]

        # Extract the displayed name from the table cell
        common_name = row.select_one("td.pl a, td.pl.custom a").text.strip()
        common_name = keep_letters_and_numbers_only_in_string(common_name)
        # Add to the dictionary
        column_name_mapping_common_as_key[common_name] = gaap_tag
        column_name_mapping_gaap_as_key[gaap_tag] = common_name

    return column_name_mapping_common_as_key, column_name_mapping_gaap_as_key

In [17]:
def create_dynamic_mapping(ticker, statement_type):
    accession_number_series = get_10K_accessionNumbers_for_ticker(ticker)
    common_as_key_mapping = defaultdict(list)
    gaap_as_key_mapping = defaultdict(list)
    for accession_number in accession_number_series:
        try:
            soup = get_statement_soup(ticker, accession_number, statement_type)
        except Exception as e:
            logging.error(
                f"Failed to get statement soup: {e} for accession number: {accession_number}"
            )
            soup = None
        if soup is None:
            pass
        else:
            mapping_common, mapping_gaap = create_column_name_mapping_to_statement(soup)
            for common_name, gaap_tag in mapping_common.items():
                common_as_key_mapping[common_name].append(gaap_tag)

            for gaap_tag, common_name in mapping_gaap.items():
                gaap_as_key_mapping[gaap_tag].append(common_name)

    final_mapping = {}

    for common_name, gaap_tags in common_as_key_mapping.items():
        if len(set(gaap_tags)) > 1:
            final_mapping[common_name] = list(set(gaap_tags))
        else:
            gaap_tag = gaap_tags[0]
            final_mapping[gaap_tag] = list(set(gaap_as_key_mapping[gaap_tag]))

    return final_mapping

In [ ]:
dynamic_mapping = create_dynamic_mapping(ticker, statement_name)

In [ ]:
columns_test, values_test = extract_columns_and_values_from_statement(soup)

In [18]:
def update_column_names_using_mapping(dynamic_mapping: dict, gaap_tags: list) -> list:
    """
    Update a list of GAAP tags based on a dynamic mapping.

    Parameters:
    - dynamic_mapping (dict): The mapping from GAAP tags to common names or vice versa.
    - gaap_tags (list): The list of GAAP tags to update.

    Returns:
    - list: Updated list of GAAP tags.
    """
    updated_gaap_tags = []

    for tag in gaap_tags:
        # If the tag matches a key, keep it as is
        if tag in dynamic_mapping:
            updated_gaap_tags.append(tag)
        else:
            # Check if the tag is a value in any of the keys
            for key, values in dynamic_mapping.items():
                if tag in values:
                    updated_gaap_tags.append(key)
                    break
            else:
                # If the tag didn't match any key or value, keep it as is
                updated_gaap_tags.append(tag)

    return updated_gaap_tags

In [ ]:
updated_columns = update_column_names_using_mapping(dynamic_mapping, columns_test)

In [19]:
def process_singular_balance_sheet(
    ticker,
    accession_number,
    statement_type,
    name_mapping: dict = None,
    first: bool = True,
):
    try:
        soup = get_statement_soup(ticker, accession_number, statement_type)
    except Exception as e:
        logging.error(
            f"Failed to get statement soup: {e} for accession number: {accession_number}"
        )
        soup = None
    if soup is None:
        pass
    else:
        columns, values_set = extract_columns_and_values_from_statement(soup)
        if name_mapping is not None:
            columns = update_column_names_using_mapping(name_mapping, columns)
        index_dates = get_two_index_dates_of_statement(soup)
        df = create_dataframe_of_statement_values_columns_dates(
            values_set, columns, index_dates
        )
        df = df.loc[:, ~df.columns.duplicated()]
        if df.empty:
            pass

        elif first == True:
            first = False
            return df

        else:
            renamed_df = df.tail(1)
            return renamed_df

In [20]:
def concatenate_all_sheets(ticker, statement_type, blank: bool = False):
    accession_number_series = get_10K_accessionNumbers_for_ticker(ticker)
    dynamic_name_mapping = create_dynamic_mapping(ticker, statement_type)
    all_sheets = pd.DataFrame()
    first = True
    for i, accession_number in enumerate(accession_number_series):
        if i > 0:
            first = False
        sheet = process_singular_balance_sheet(
            ticker,
            accession_number,
            statement_type,
            name_mapping=dynamic_name_mapping,
            first=first,
        )
        all_sheets = pd.concat([all_sheets, sheet], axis=0, join="outer")
        print(f"concatenated {statement_type} {i} to the full Dataframe")
    if blank == True:
        blank_all_sheets = all_sheets.replace(["None", np.NaN], "")
        return blank_all_sheets.T
    else:
        return all_sheets.T

In [14]:
single_tesing_df = process_singular_balance_sheet(
    ticker, accession_number, statement_name, name_mapping=dynamic_mapping, first=True
)

In [21]:
all_sheets = concatenate_all_sheets(ticker, statement_name, blank=True)

ERROR:root:Failed to get statement soup: 'consolidated balance sheets' for accession number: 000119312510075151


concatenated consolidated balance sheets 0 to the full Dataframe
concatenated consolidated balance sheets 1 to the full Dataframe
concatenated consolidated balance sheets 2 to the full Dataframe
concatenated consolidated balance sheets 3 to the full Dataframe
concatenated consolidated balance sheets 4 to the full Dataframe
concatenated consolidated balance sheets 5 to the full Dataframe
concatenated consolidated balance sheets 6 to the full Dataframe
concatenated consolidated balance sheets 7 to the full Dataframe
concatenated consolidated balance sheets 8 to the full Dataframe
concatenated consolidated balance sheets 9 to the full Dataframe
concatenated consolidated balance sheets 10 to the full Dataframe
concatenated consolidated balance sheets 11 to the full Dataframe
concatenated consolidated balance sheets 12 to the full Dataframe


ERROR:root:Failed to get statement soup: 'consolidated balance sheets' for accession number: 000119312510075151


concatenated consolidated balance sheets 13 to the full Dataframe


In [22]:
all_sheets

,2023-01-29,2022-01-30,2021-01-31,2020-02-02,2019-02-03,2018-01-28,2017-01-29,2016-01-31,2015-02-01,2014-02-02,2013-02-03,2012-01-29,2011-01-30
us-gaap_AssetsCurrentAbstract,,,,,,,,,,,,,
us-gaap_CashAndCashEquivalentsAtCarryingValue,367344,850338,1200337,432162,338954,390136,213713,193647,222927,330121,424555,502757,628403000
us-gaap_ReceivablesNetCurrent,115685,131683,143728,111737,107102,90119,88803,79304,67465,60330,62985,45961,41565000
us-gaap_InventoryNet,1456123,1246372,1006299,1100544,1124992,1061593,977505,978138,887701,813160,640024,553461,513381000
us-gaap_PrepaidExpenseCurrent,64961,69252,93822,90426,101356,62204,52882,44654,36265,35309,26339,24188,21120000
us-gaap_OtherAssetsCurrent,31967,26249,22894,20766,21939,11876,10652,11438,13005,10852,9819,9229,8176000
us-gaap_AssetsCurrent,2036080,2323894,2467080,1755635,1694343,1636445,1367180,1336100,1391923,1419103,1316772,1276366,1347594000
us-gaap_PropertyPlantAndEquipmentNet,1065381,920773,873894,929038,929635,932283,923283,886813,883012,849293,812037,734672,730556000
us-gaap_OperatingLeaseRightOfUseAsset,1286452,1132764,1086009,1166383,,,,,,,,,
Deferred income taxes net,81389,56585,61854,47977,,,,,,121486,99764,91744,85612000


In [ ]:
def create_many_to_one_name_mapping_to_statement(soup: BeautifulSoup) -> dict:
    """
    Modified to create a many-to-one dictionary. still need to implement
    """
    column_name_mapping = defaultdict(list)
    for row in soup.select("tr.re, tr.ro, tr.reu, tr.rou"):
        onclick_attr = row.select_one("td.pl a, td.pl.custom a")["onclick"]
        column_title = onclick_attr.split("defref_")[-1].split("',")[0]
        displayed_name = row.select_one("td.pl a, td.pl.custom a").text.strip()
        displayed_name = keep_letters_and_numbers_only_in_string(displayed_name).lower()
        column_name_mapping[displayed_name].append(column_title)
    return column_name_mapping

In [ ]:
def rename_df_columns_using_mapping(
    df: pd.DataFrame, column_name_mapping: dict
) -> pd.DataFrame:
    """
    Rename columns in the DataFrame using the many-to-one mapping dictionary.

    Parameters:
    - df (pd.DataFrame): The input DataFrame.
    - column_name_mapping (dict): The many-to-one mapping dictionary.

    Returns:
    - pd.DataFrame: DataFrame with renamed columns.
    """
    # Create a reverse mapping from internal column title to displayed name
    reverse_mapping = {
        col: disp_name
        for disp_name, cols in column_name_mapping.items()
        for col in cols
    }
    # Rename columns in the DataFrame
    df = df.rename(columns=reverse_mapping)

    return df